In [1]:
import asyncio
import json
import logging

from signalr_async.net import Hub
from signalr_async.net.client import SignalRClient

import merger
# from scoreboard import Scoreboard

logging.basicConfig(level=logging.DEBUG)
# s = Scoreboard()


class F1SignalRClient(SignalRClient):
    driver_list = {}
    timing_data = {}
    session_data = {}
    session_info = {}

    def LapCount(self, lc):
        pass

    def SessionData(self, session_data):
        merger.merge(self.session_data, session_data)

    def SessionInfo(self, session_info):
        merger.merge(self.session_info, session_info)

    def DriverList(self, drivers):
        merger.merge(self.driver_list, drivers)

    def TimingData(self, timing):
        merger.merge(self.timing_data, timing)

        for driver_number in timing["Lines"]:
            driver = self.driver_list[driver_number]
            driver_timing = self.timing_data["Lines"][driver_number]
            gap = ''
            color = driver['TeamColour']
            pos = driver_timing['Position']
            if not color:
                color = 'ffffff'

            if self.session_info['Type'] == "Qualifying":
                current_part = int(self.timing_data['SessionPart'])
                current_entries = self.timing_data['NoEntries'][current_part - 1]
                if driver_timing['Position'] == '1':
                    gap = driver_timing['BestLapTimes'][current_part - 1]['Value']
                elif int(driver_timing['Position']) > current_entries:
                    gap = 'KO'
                else:
                    gap = driver_timing['Stats'][current_part - 1]['TimeDiffToFastest']
            elif self.session_info['Type'] == "Practice":
                if 'TimeDiffToFastest' in driver_timing:
                    gap = driver_timing['TimeDiffToFastest']
                if driver_timing['Position'] == '1':
                    gap = driver_timing['BestLapTime']['Value']
            elif self.session_info['Type'] == "Race":
                if 'GapToLeader' in driver_timing:  # race
                    print("GapToLeader")
                    gap = driver_timing['GapToLeader']

            # s.scoreboard[driver_number] = {
            #     "Pos": pos,
            #     "TLA": driver['Tla'],
            #     "Gap": gap,
            #     "Color": (int(color[:2], 16), int(color[2:4], 16), int(color[4:], 16)),
            # }

    async def _process_message(self, message):
        if hasattr(message, "result"):
            print(message)
            for k in message.result:
                print(k)
                if hasattr(self, k):
                    getattr(self, k)(message.result[k])
                else:
                    print(k, message.result[k])
        elif hasattr(message, "arguments"):
            k, v, t = message.arguments
            if hasattr(self, k):
                getattr(self, k)(v)
            else:
                print(k, v)


_connection_url = 'https://livetiming.formula1.com/signalr'
hub = Hub("streaming")


async def run_client():
    async with F1SignalRClient(
            _connection_url,
            [hub],
            keepalive_interval=5,
    ) as client:
        await hub.invoke("Subscribe", ["Heartbeat", "CarData.z", "Position.z",
                       "ExtrapolatedClock", "TopThree", "RcmSeries",
                       "TimingStats", "TimingAppData",
                       "WeatherData", "TrackStatus", "DriverList",
                       "RaceControlMessages", "SessionInfo",
                       "SessionData", "LapCount", "TimingData"])
        await client.wait(timeout=5)



In [2]:
# https://github.com/iebb/PixooLiveTiming

await asyncio.gather(
        run_client(),
        # s.heartbeat()
    )


DEBUG:signalr_async.client:Starting connection with disconnected state
DEBUG:signalr_async.client:Negotiation started
DEBUG:signalr_async.client:Negotiation response: {'Url': '/signalr', 'ConnectionToken': 'G/MpRHIkt5CfupPe4/G9Fe4qGqxXKmHG8ZITD7Tpz2yAT604ffmOvTdjTJAjQPQiOpkr3NmRm3oPd3+HHFz5pz9Syi8jCob2xat5oCCicwmrEEXx25fR/r1F1DY4PJdQ', 'ConnectionId': '5f29b661-9465-4562-868b-0a2e290f2f90', 'KeepAliveTimeout': 20.0, 'DisconnectTimeout': 30.0, 'ConnectionTimeout': 110.0, 'TryWebSockets': True, 'ProtocolVersion': '1.5', 'TransportConnectTimeout': 10.0, 'LongPollDelay': 1.0}
DEBUG:signalr_async.client:Connecting to wss://livetiming.formula1.com/signalr/connect?clientProtocol=1.5&connectionData=%5B%7B%22name%22:+%22streaming%22%7D%5D&transport=webSockets&connectionToken=G/MpRHIkt5CfupPe4/G9Fe4qGqxXKmHG8ZITD7Tpz2yAT604ffmOvTdjTJAjQPQiOpkr3NmRm3oPd3%2BHHFz5pz9Syi8jCob2xat5oCCicwmrEEXx25fR/r1F1DY4PJdQ
DEBUG:signalr_async.client:Initializing
DEBUG:signalr_async.client:Raw message received: WSM

HubResult(invocation_id='0', result={'Heartbeat': {'Utc': '2023-03-19T16:38:05.6876071Z', '_kf': True}, 'CarData.z': '7ZS9DsIwDITfxXOp4vyarBVvAAuIoUKVQEIdoFvVd6dNZwZuoYMXJ4p8kn0XfSMd+uH16N6ULyOdhhtlssa6nXE73h85ZieZXS1WxMj+TBU17WvuHomX0tzbvu+e5cFQNhXZUl2pvtSw3pdjmkoDpPOgjs3vQi5CdEOGR43oqIKGgZpj4fjR/K1DhUAc644JFDrUHI/+nBDAUSPqakLNEcycWfodUL5mkRS8AkoBpYBSQG0QUMFLMlb5pHxSPimftsenJBJjTAooBZQCSgH1F0Bdpw8=', 'Position.z': '7ZY9a8NADIb/i2Y7WNJ975lbqIc2pYMpGUyJU2J3Mv7v9UcI55Kq1PMtxoZ7kO6VeHAPj+e27upzA+G1h7I+HduuOn1CACqI84Jz9CWawC4g75xXZBQdIIN9013qYwuhB5weT13VfY2f8NCUl+r9YzzyDCFH9iaDFwi6UBkcICC6IQOSEPR2RnIdI0pCyOkZoQIjBAuJUYxLZ97FzIbboNibYjczBm3MGLFOsdQhverNiYxa6rCNMyAxA2K+3idG5AhwQcjG1yFxojxNcmrNr+qw2Bpd66wQMelfErBiGe1nZnyJGBYjuL85Su7t7kS1FnszC4Mq3jbz/9isGAFPZydkWu4b48QISC2M8TdkGLK/DKJ244xo3JxkkGSQZJBkkA0GYavQY/oHSQZJBkkG2WIQqzUyczJIMkgySDLIT4O8Dd8=', 'ExtrapolatedClock': {'Utc': '2023-03-19T16:38:15.6399535Z', 'Remaining': '02:00:00', 'Extrapolating': False, '_kf': True}, 'TopThree': {'Withhel

DEBUG:signalr_async.client:Raw message received: WSMessage(type=<WSMsgType.TEXT: 1>, data='{"C":"d-F23263D5-B,0|Df,0|Dg,11|d,94|p,89C|o,89B|a,1|Y,D|Dh,0|n,1|W,2|c,25|i,1|X,C|k,3|l,1|r,2|Z,1|s,59F","M":[{"H":"Streaming","M":"feed","A":["CarData.z","7ZS9CsMwDITfRXMSLP/ba+gbtEtLh1ACLZQMbbaQd2/izB16SzNokY3RgXRnvokOw/h69G/Kl4lO440yaaVNrUzN6cg+m5jZNSEmxyGeqaK2ey3dE/Fa2ns3DP2zPCjKqiJdqinVluq2+3rMc2mAdBbUsfpdyEWIbsjwqB4dNaJhoOZoOH40f21QIRDHtmMAhQY1x6I/xzlwVI+6GlBzImbOIv0OKN8on2xwVgAlgBJACaD2Byirk9WaBVACKAGUAGp/gAoqmaiCAEoAJYASQP0FUNf5Aw==","2023-03-19T16:38:16.7093807Z"]},{"H":"Streaming","M":"feed","A":["Position.z","7ZQ9a8MwEIb/y8120OlOn3vmFpqhTekQSgZTkpTEnYL/e/0RglLSK/V8i5FBD/fqlXjO8Hg4NW1z2EN+PcOq2W1P7Wb3CRmssVQbqjGt0GeKGd0iJE7B2TVUsNy3x2Z7gnwGHD5P7ab96n/hYb86bt4/+i3PkGuk5Ct4gewMV7CGjBi7CqyEYAojUrsSYQmx0Y2INVggaCSGCadkKZbMjNOgmI0pjozHUDJenGOmOdbdZIsiw9McCmUHVuzAEl3OUyJyBTghNpTHseKN0nCTQ7R0M4fEaPYy5wYRm/6lgSCOcWlk+kXBkFjB/ZfDcra7N+qcmM1PDHL52vz/awtiBTTsHZDhcV+ZKFZgeWJ8uiJdV/1lEL8wxIliUIOoQd

CarData.z 7ZS9CsMwDITfRXMSLP/ba+gbtEtLh1ACLZQMbbaQd2/izB16SzNokY3RgXRnvokOw/h69G/Kl4lO440yaaVNrUzN6cg+m5jZNSEmxyGeqaK2ey3dE/Fa2ns3DP2zPCjKqiJdqinVluq2+3rMc2mAdBbUsfpdyEWIbsjwqB4dNaJhoOZoOH40f21QIRDHtmMAhQY1x6I/xzlwVI+6GlBzImbOIv0OKN8on2xwVgAlgBJACaD2Byirk9WaBVACKAGUAGp/gAoqmaiCAEoAJYASQP0FUNf5Aw==
Position.z 7ZQ9a8MwEIb/y8120OlOn3vmFpqhTekQSgZTkpTEnYL/e/0RglLSK/V8i5FBD/fqlXjO8Hg4NW1z2EN+PcOq2W1P7Wb3CRmssVQbqjGt0GeKGd0iJE7B2TVUsNy3x2Z7gnwGHD5P7ab96n/hYb86bt4/+i3PkGuk5Ct4gewMV7CGjBi7CqyEYAojUrsSYQmx0Y2INVggaCSGCadkKZbMjNOgmI0pjozHUDJenGOmOdbdZIsiw9McCmUHVuzAEl3OUyJyBTghNpTHseKN0nCTQ7R0M4fEaPYy5wYRm/6lgSCOcWlk+kXBkFjB/ZfDcra7N+qcmM1PDHL52vz/awtiBTTsHZDhcV+ZKFZgeWJ8uiJdV/1lEL8wxIliUIOoQdQgapAZBrG9QUwkNYgaRA2iBplhEHYcAyc1iBpEDaIGmWEQ3xuE2alB1CBqEDXIT4O8dd8=


DEBUG:signalr_async.client:Raw message received: WSMessage(type=<WSMsgType.TEXT: 1>, data='{"C":"d-F23263D5-B,0|Df,0|Dg,11|d,94|p,89C|o,89C|a,1|Y,D|Dh,0|n,1|W,2|c,25|i,1|X,C|k,3|l,1|r,2|Z,1|s,59F","M":[{"H":"Streaming","M":"feed","A":["Position.z","7Ze7asNAEEX/ZWrJaB777F0nEBWJQwoRXIhgOdhKZfTv0cOYdXAmxPU2QoI9zN27ywGd4HF/bPt230F8PUHd7rbHvtl9QgSqiMuKSww12sg+ol15Fo8ON1DAuusP7fYI8QQ4PZ76pv8aP+Ghqw/N+8e45BliiRxsAS8QTSUFbCAi+qEA0hAMbkZKkyKiIeTNjFCFCYKVxgjjkiz4lLljN6hmE/YzY9GljFXnVMscMlfZvMrIModd2gGpHRDzeT8poleAC0Iu3Q6pJ8rTSU7RwtUcVqPRec4Vojb9SwNOHWPCzIwvCcNqBbdvjujZbp6oMWo2uzAo6W2z/6/NqRXwtHZCpst9YbxaAcnC2HBBhqH4yyBuRUacYc4GyQbJBskGucMgguIoZIFkgWSBZIHcIRCHYscfmWyQbJBskGyQnwZ5G74B","2023-03-19T16:38:17.7146834Z"]}]}', extra='')
DEBUG:signalr_async.client:Raw message received: WSMessage(type=<WSMsgType.TEXT: 1>, data='{"C":"d-F23263D5-B,0|Df,0|Dg,11|d,94|p,89D|o,89C|a,1|Y,D|Dh,0|n,1|W,2|c,25|i,1|X,C|k,3|l,1|r,2|Z,1|s,59F","M":[{"H":"Streaming","M":"feed","A":["CarData.z","7ZS9CkIxDIXfJbNK07Tpzy

...

CarData.z 7Ze9isMwDIDfRXNbJMvy31r6Br3ljg6lFO7gyNDrFvLuTdXhltAEjY4XORh/RJLFR9LDobvffq5/UL56+LhfoIBDx1vkLeUjheJd4bhjzMH5/Akb2J9v4+ke6Bn23+euu/7qBkLBDTiNrNFrlNfzcxkGPWDivJEjnABJYnjPkrLWIsmcbVgEEnpFdf1PN03AHiktuZapNs1krC915kGwToJjK2i4lVeN0QiytTneOkAixlSDtatxWXOmRzbZGjSi73QlmCULV6Sr5FajK5kptemq6aouXcVRV0FiTbqi1egqzXxINl01XdWkq7RDl4Wir0lXa/kZZJTYdNV0VZmuTsMD
Position.z 7ZRNa4NAEIb/y5w1ODP7fe+5hXpoU3oIJQcpMSXak/jfq24Ia0mnJOe9iMI+zLvvrs8AT8eu6ZtjC+FtgLo57Lt+d/iCAFQRlxWX6Gs0QVFgu2GriB1uoYCHtj81+w7CADg/nvtd/z19wmNbn3Yfn9OSFwglsjcFvELQlSpgCwHRjQWQhKC3C1JqnSBKQsjpBaEKEwQriVGMMZl3KXPHblDMptgtjEGbMkacU8U5pFfZnMioOIdt2gGJHRDzeT8pIleAESGbbofEE2WtYjS/msNiNDrPWSFi0380YMUx2i/M9JIwLFZw/eYoOdvVE9VazGYigyq9beb22qxYAc9rZ2S+3BfGiRWQiozxF2Qci/8NYlBN/6jNBskGyQbJBrnDIJ41avLZINkg2SDZIDcbxG3Qa0TmbJBskGyQbJDfBnkffwA=


DEBUG:signalr_async.client:Raw message received: WSMessage(type=<WSMsgType.TEXT: 1>, data='{"C":"d-F23263D5-B,0|Df,0|Dg,11|d,A6|p,99E|o,99D|a,1|Y,D|Dh,0|n,1|W,2|c,29|i,1|X,C|k,3|l,1|r,2|Z,1|s,59F","M":[{"H":"Streaming","M":"feed","A":["CarData.z","nZKxDsIwDET/xXOL7DhN26wVfwALiKFClUBCHUq3KP9e6g4sDOWWcxT5Sfb5Eh3HeXoOb4rXROf5TpEcOy1ZS2lPEqJ3UdsDK2to9EIFdf306U4kq3SPfhyHl30wRS7ImaqpN62291pytgaI8yAn/D8oBqIbCjxq2AUKe0Otfsdt0IOgBjk4AmgGnKIgcJJtxxoEFTXHo+mpKnDUgLpa7zPnd14bzKCcb3kB","2023-03-19T16:42:39.0303683Z"]},{"H":"Streaming","M":"feed","A":["Position.z","7ZQ/S8RAEMW/y9SJ7Mzs//5qBVPoiUWQK4JcTi6xCvnuJtnj2Mg5onWasIH98d68Hd4AD6eu6ZtTC/FlgKo5Hrq+Pn5ABFLEpeISQ4U2aooc7hTr4I3bQwG7tj83hw7iADh/Hvu6/5x+4b6tzvXb+3TlCWKJHGwBzxCN0gXsISL6sQCSEAxuQUpjMkRLCHmzIKQwQ1BJjGZMzoLPmX9Mg6I3zX5hLLqcsaKOSjpkVt68yOikwy7PgMQMiPkyT47IEWBCyOXjkPiibHSyFlY6LFqji84KEZP+IQEnypiwMNMhY1iM4PbmaNnbzRc1RvRmE4M63zb799icGAHPd2dkXu4r48UISCfGhisyjsXvDUKog/G8NcjWIFuDbA3yvUFexy8=","2023-03-19T16:42:39.2149583Z"]}]}', extra='')


CarData.z nZKxDsIwDET/xXOL7DhN26wVfwALiKFClUBCHUq3KP9e6g4sDOWWcxT5Sfb5Eh3HeXoOb4rXROf5TpEcOy1ZS2lPEqJ3UdsDK2to9EIFdf306U4kq3SPfhyHl30wRS7ImaqpN62291pytgaI8yAn/D8oBqIbCjxq2AUKe0Otfsdt0IOgBjk4AmgGnKIgcJJtxxoEFTXHo+mpKnDUgLpa7zPnd14bzKCcb3kB
Position.z 7ZQ/S8RAEMW/y9SJ7Mzs//5qBVPoiUWQK4JcTi6xCvnuJtnj2Mg5onWasIH98d68Hd4AD6eu6ZtTC/FlgKo5Hrq+Pn5ABFLEpeISQ4U2aooc7hTr4I3bQwG7tj83hw7iADh/Hvu6/5x+4b6tzvXb+3TlCWKJHGwBzxCN0gXsISL6sQCSEAxuQUpjMkRLCHmzIKQwQ1BJjGZMzoLPmX9Mg6I3zX5hLLqcsaKOSjpkVt68yOikwy7PgMQMiPkyT47IEWBCyOXjkPiibHSyFlY6LFqji84KEZP+IQEnypiwMNMhY1iM4PbmaNnbzRc1RvRmE4M63zb799icGAHPd2dkXu4r48UISCfGhisyjsXvDUKog/G8NcjWIFuDbA3yvUFexy8=


DEBUG:signalr_async.client:Stopping connection with connected state
DEBUG:signalr_async.client:Websocket closed
DEBUG:signalr_async.client:Session closed
DEBUG:signalr_async.client:gather=[None, None, None, None]


CancelledError: 

In [5]:
F1SignalRClient.driver_list

{'11': {'RacingNumber': '11',
  'BroadcastName': 'S PEREZ',
  'FullName': 'Sergio PEREZ',
  'Tla': 'PER',
  'Line': 1,
  'TeamName': 'Red Bull Racing',
  'TeamColour': '3671C6',
  'FirstName': 'Sergio',
  'LastName': 'Perez',
  'Reference': 'SERPER01',
  'CountryCode': 'MEX',
  'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/S/SERPER01_Sergio_Perez/serper01.png.transform/1col/image.png'},
 '14': {'RacingNumber': '14',
  'BroadcastName': 'F ALONSO',
  'FullName': 'Fernando ALONSO',
  'Tla': 'ALO',
  'Line': 2,
  'TeamName': 'Aston Martin',
  'TeamColour': '358C75',
  'FirstName': 'Fernando',
  'LastName': 'Alonso',
  'Reference': 'FERALO01',
  'CountryCode': 'ESP',
  'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png'},
 '63': {'RacingNumber': '63',
  'BroadcastName': 'G RUSSELL',
  'FullName': 'George RUSSELL',
  'Tla': 'RUS',
  'Line': 3,
  'TeamName': 'Mercedes',
  'TeamCol

In [6]:
F1SignalRClient.timing_data

{'Lines': {'11': {'GapToLeader': 'LAP 1',
   'IntervalToPositionAhead': {'Value': 'LAP 1', 'Catching': False},
   'Line': 1,
   'Position': '1',
   'ShowPosition': True,
   'RacingNumber': '11',
   'Retired': False,
   'InPit': False,
   'PitOut': False,
   'Stopped': False,
   'Status': 64,
   'Sectors': [{'Stopped': False,
     'Value': '',
     'Status': 0,
     'OverallFastest': False,
     'PersonalFastest': False,
     'Segments': [{'Status': 2064},
      {'Status': 2064},
      {'Status': 2064},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048}]},
    {'Stopped': False,
     'Value': '',
     'Status': 0,
     'OverallFastest': False,
     'PersonalFastest': False,
     'Segments': [{'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048},
      {'Status': 2048}]},
    {'Stopped': False,
     'Value': '',
   

In [7]:
F1SignalRClient.session_data

{'Series': [{'Utc': '2023-03-19T16:01:05.191Z', 'Lap': 1}],
 'StatusSeries': [{'Utc': '2023-03-19T16:00:41.957Z',
   'TrackStatus': 'AllClear'}],
 '_kf': True}

In [8]:
F1SignalRClient.session_info

{'Meeting': {'Key': 1142,
  'Name': 'Saudi Arabian Grand Prix',
  'OfficialName': 'FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023',
  'Location': 'Jeddah',
  'Country': {'Key': 153, 'Code': 'KSA', 'Name': 'Saudi Arabia'},
  'Circuit': {'Key': 149, 'ShortName': 'Jeddah'}},
 'ArchiveStatus': {'Status': 'Generating'},
 'Key': 7779,
 'Type': 'Race',
 'Name': 'Race',
 'StartDate': '2023-03-19T20:00:00',
 'EndDate': '2023-03-19T22:00:00',
 'GmtOffset': '03:00:00',
 'Path': '2023/2023-03-19_Saudi_Arabian_Grand_Prix/2023-03-19_Race/',
 '_kf': True}